In [6]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time  # time モジュールを追加

# DBの初期化（既存のテーブルをリセット）
conn = sqlite3.connect('課題1.db')
conn.execute('DROP TABLE IF EXISTS repositories')
conn.commit()
print("既存のテーブルをリセットしました")

# テーブル作成 - カラム名を日本語に変更
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS repositories (リポジトリ名 TEXT, 主要な言語 TEXT, スターの数 INTEGER)')
conn.commit()


既存のテーブルをリセットしました


In [7]:

# 0.5秒スリープを追加
time.sleep(0.5)
# GitHubからデータ取得
url = "https://github.com/google?tab=repositories"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
res = requests.get(url, headers=headers, timeout=15)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.text, "html.parser")

print(f"ステータスコード：{res.status_code}")

# リポジトリ情報の抽出
repos_data = []
repos = soup.select('li.Box-row')

for repo in repos:
    # リポジトリ名
    name_elem = repo.select_one('a[itemprop="name codeRepository"]')
    repo_name = name_elem.text.strip() if name_elem else "不明"
    
    # メイン言語
    lang_elem = repo.select_one('span[itemprop="programmingLanguage"]')
    main_language = lang_elem.text.strip() if lang_elem else "不明"
    
    # スター数
    stars = 0
    stars_elem = repo.select_one('a[href$="/stargazers"]')
    if stars_elem:
        stars_text = stars_elem.text.strip()
        # カンマを削除して数字だけを抽出
        digits_only = ''.join(c for c in stars_text if c.isdigit())
        if digits_only:
            stars = int(digits_only)
    
    repos_data.append((repo_name, main_language, stars))

# データベースに保存
if repos_data:
    cur.executemany('INSERT INTO repositories VALUES (?, ?, ?)', repos_data)
    conn.commit()
    print(f"{len(repos_data)}件のリポジトリ情報を保存しました")
else:
    print("リポジトリ情報が取得できませんでした")

# 結果の表示
cur.execute('SELECT * FROM repositories ORDER BY スターの数 DESC')
print("\n===== 保存したリポジトリ情報 =====")
for row in cur.fetchall():
    print(f"リポジトリ: {row[0]}, 言語: {row[1]}, スター: {row[2]}")

conn.close()

ステータスコード：200
10件のリポジトリ情報を保存しました

===== 保存したリポジトリ情報 =====
リポジトリ: differential-privacy, 言語: Go, スター: 3253
リポジトリ: neuroglancer, 言語: TypeScript, スター: 1230
リポジトリ: meridian, 言語: Python, スター: 1189
リポジトリ: quiche, 言語: C++, スター: 801
リポジトリ: dawn, 言語: C++, スター: 780
リポジトリ: android-cuttlefish, 言語: C++, スター: 573
リポジトリ: orbax, 言語: Python, スター: 456
リポジトリ: nsscache, 言語: Python, スター: 150
リポジトリ: device-infra, 言語: Java, スター: 58
リポジトリ: netkat, 言語: C++, スター: 28
